In [2]:
import cv2
import numpy as np
from keras.models import load_model
from keras.preprocessing.image import load_img


In [3]:
# Load the Keras model
loaded_model = load_model("emotionCheck.keras")

label = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [4]:
# Function to preprocess the input image
def preprocess_image(image_path):
    img = load_img(image_path, color_mode="grayscale")
    feature = np.array(img)
    feature = feature.reshape(1, 48, 48, 1)
    return feature / 255.0

In [5]:
# Function for real-time face recognition
def real_time_face_recognition():
    # Open a connection to the camera (0 is usually the built-in camera)
    cap = cv2.VideoCapture(0)

    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()

        # Convert the frame to grayscale for face detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Perform face detection using Haar cascades
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

        for (x, y, w, h) in faces:
            # Extract the face region
            face_roi = gray[y:y + h, x:x + w]

            # Resize the face image to match the input size of your model
            face_roi = cv2.resize(face_roi, (48, 48))

            # Preprocess the face image
            face_input = face_roi.reshape(1, 48, 48, 1) / 255.0

            # Make a prediction using the loaded model
            emotion_probabilities = loaded_model.predict(face_input)
            predicted_emotion = label[np.argmax(emotion_probabilities)]

            # Draw a rectangle around the face and display the predicted emotion
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            cv2.putText(frame, f"Emotion: {predicted_emotion}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

        # Display the resulting frame
        cv2.imshow('Real-time Face Recognition', frame)

        # Break the loop when 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the camera and close all windows
    cap.release()
    cv2.destroyAllWindows()

In [6]:
# Run the real-time face recognition function
real_time_face_recognition()

1/1 [==============================] - 0s 18ms/step
